In [163]:
import numpy as np
import pandas as pd
import pathlib
from tqdm.auto import tqdm

import ray

In [164]:
base_path = "/home/jakobs"

In [165]:
data_baseline = pd.read_feather(f"{base_path}/data/data_baseline_230321.feather")

In [166]:
data_baseline

,eid,ml_glaucoma_gradability_grad_l,ml_glaucoma_gradability_grad_r,ml_glaucoma_gradability_wdiff_l,ml_glaucoma_gradability_wdiff_r,age3,alcohol3,cigstat3,recruitment_date,systol3,diastol3,mi3,cva3,sex,bmi3,cholesterol3,triglyceride3,hdl3,ldl3
0,RID00001,0.955591,0.760914,0.043725,0.226360,86.127310,2.0,3.0,2007-08-07,138.0,72.5,0.0,0.0,2,33.455266,6.4,1.5,1.53,4.27
1,RID00002,0.986005,0.943931,0.013981,0.054606,70.505133,2.0,3.0,2010-07-08,105.5,68.5,0.0,0.0,2,25.593164,3.9,2.3,1.50,1.40
2,RID00003,0.990022,0.981540,0.009343,0.017813,64.577687,1.0,2.0,2007-04-26,128.5,78.5,0.0,0.0,2,25.372781,3.9,2.1,1.52,1.48
3,RID00004,0.894128,0.945897,0.103611,0.052175,66.740589,1.0,3.0,2006-07-31,154.5,87.5,0.0,0.0,2,27.070312,6.0,1.3,1.80,3.70
4,RID00006,0.940763,0.959411,0.055711,0.039147,72.366872,2.0,3.0,2007-07-10,138.0,66.5,0.0,0.0,2,26.189022,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7243,RID07407,0.940662,0.962702,0.058866,0.037013,73.675565,1.0,3.0,2006-11-03,132.0,73.0,0.0,0.0,1,25.032679,3.9,1.2,1.39,2.01
7244,RID07408,0.719619,0.637359,0.276180,0.355266,64.533881,1.0,3.0,2008-12-18,147.5,81.0,0.0,0.0,2,27.957894,6.8,1.0,1.76,4.64
7245,RID07409,0.635806,0.926931,0.342445,0.072251,82.475017,NaN,NaN,2007-09-27,144.5,80.0,0.0,0.0,2,26.053959,6.4,1.5,1.95,3.85
7246,RID07410,0.861651,0.054378,0.133713,0.028145,82.568104,1.0,2.0,2010-07-21,109.5,63.5,0.0,0.0,1,29.726252,NaN,NaN,NaN,NaN


## Map records to phecodes

In [167]:
endpoints_md = pd.read_csv(f"{base_path}/BiHealth/onnx/endpoints.csv", dtype={"phecode":str}).drop(columns="Unnamed: 0").set_index("endpoint")#[["endpoint", "eligable", "n", "freq", "phecode", "phecode_string", "phecode_category", "sex"]]
endpoints_md

,eligable,n,freq,phecode,phecode_string,phecode_category,sex
endpoint,,,,,,,
OMOP_4306655,61256,3548,0.057921,4306655.0,All-Cause Death,Death,Both
phecode_002,60945,658,0.010797,2.0,Staphylococcus,ID,Both
phecode_002-1,61010,486,0.007966,2.1,Staphylococcus aureus,ID,Both
phecode_003,60757,1017,0.016739,3.0,Escherichia coli,ID,Both
phecode_004,60584,494,0.008154,4.0,Streptococcus,ID,Both
...,...,...,...,...,...,...,...
phecode_977-52,31669,520,0.016420,977.52,Hormone replacement therapy (postmenopausal),Rx,Female
phecode_977-7,60032,2231,0.037164,977.7,Long term (current) use of insulin or oral hyp...,Rx,Both
phecode_977-71,60936,472,0.007746,977.71,Long term (current) use of insulin,Rx,Both


In [168]:
phecodes_md = pd.read_csv(f"{base_path}/mapping/phecode_strings_V2.csv", dtype={"phecode":str, "icd10":str}).sort_values("phecode").reset_index(drop=True)#.rename(columns={"icd10":"icd"})[["icd", "phecode"]]
phecodes_md["phecode"] = phecodes_md["phecode"].str.replace(".", "-")
phecodes_md

/tmp/ipykernel_60255/1553263133.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  phecodes_md["phecode"] = phecodes_md["phecode"].str.replace(".", "-")


,phecode,phecode_string,phecode_category,sex,ICD10_only,phecode_top,leaf
0,001,Salmonella,ID,Both,0,1,1
1,002,Staphylococcus,ID,Both,0,2,0
2,002-1,Staphylococcus aureus,ID,Both,0,2,1
3,003,Escherichia coli,ID,Both,0,3,1
4,004,Streptococcus,ID,Both,0,4,0
...,...,...,...,...,...,...,...
3657,992,Family history of diseases of the skin and sub...,Stat,Both,0,992,1
3658,993,Family history of musculoskeletal disease,Stat,Both,0,993,1
3659,994,Family history of congenital anomalies,Stat,Both,0,994,1
3660,995,Family history of genetic condition,Stat,Both,0,995,1


In [169]:
icd10_phecode_mapping = pd.read_csv(f"{base_path}/mapping/ICD10_to_phecode_V2.csv", dtype={"phecode":str, "icd10":str}).rename(columns={"icd10":"icd"})[["icd", "phecode"]]
icd10_phecode_mapping["phecode"] = icd10_phecode_mapping["phecode"].str.replace(".", "-")
icd10_phecode_mapping

/tmp/ipykernel_60255/113536806.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  icd10_phecode_mapping["phecode"] = icd10_phecode_mapping["phecode"].str.replace(".", "-")


,icd,phecode
0,A00,023
1,A00,089
2,A00,089-1
3,A00,526
4,A00,526-1
...,...,...
86915,Z98.61,436
86916,Z98.62,436
86917,Z98.62,436-2
86918,Z99.2,584


## Create Metdata and load records

In [170]:
hes = pd.read_stata(f"{base_path}/BiHealth/Data/StudyData/HESLong2019HC3.dta").rename(columns={"bihid": "eid", "admidate":"date", "Diag":"icd"})[["eid", "icd", "date"]].assign(source="HES")

In [171]:
deaths = pd.read_stata(f"{base_path}/BiHealth/Data/StudyData/DeathsLong2019HC3.dta").rename(columns={"bihid": "eid", "dod":"date", "diag":"icd"})[["eid", "icd", "date"]].assign(source="ONS")

In [172]:
metadata_md = data_baseline[["eid", "recruitment_date"]]\
    .merge(deaths[["eid", "date"]].drop_duplicates().rename(columns={"date":"death_date"}), how="left")\
    .assign(censoring_date=pd.to_datetime("2019-06-17", infer_datetime_format=True))#\
    #.assign(exit_date = lambda x: min(x.date, x.censoring_date))[["eid", "recruitment_date", "exit_date"]]
metadata_md["exit_date"] = metadata_md["death_date"].combine_first(metadata_md["censoring_date"])
metadata_md = metadata_md[["eid", "recruitment_date", "death_date", "exit_date"]].assign(exit_time=lambda x: (x.exit_date - x.recruitment_date).dt.days / 365.25)

In [173]:
icd_long = pd.concat([hes, deaths], axis=0).sort_values(["eid", "date"]).drop_duplicates().reset_index(drop=True)

In [174]:
phecodes_long = icd_long.merge(icd10_phecode_mapping, how="left", on="icd")
phecodes_long

,eid,icd,date,source,phecode
0,RID00001,L82X,2000-12-15,HES,NaN
1,RID00001,M4712,2001-03-29,HES,NaN
2,RID00001,M4712,2001-05-13,HES,NaN
3,RID00001,G992,2001-05-13,HES,NaN
4,RID00001,M4712,2001-05-18,HES,NaN
...,...,...,...,...,...
190465,RID07411,C189,2018-08-21,ONS,NaN
190466,RID07411,F03,2018-08-21,ONS,328
190467,RID07411,F03,2018-08-21,ONS,328-9
190468,RID07411,C509,2018-08-21,ONS,NaN


In [175]:
phecodes_long_mapped_native = phecodes_long.query("phecode==phecode")
phecodes_long_unmapped_hack = phecodes_long.query("phecode!=phecode").drop(columns="phecode").assign(icd= lambda x: x.icd.str.slice(0, 3)).merge(icd10_phecode_mapping, how="left", on="icd")
phecodes_long_mapped = pd.concat([phecodes_long_mapped_native, phecodes_long_unmapped_hack], axis=0).drop_duplicates().sort_values(["eid", "date"]).query("phecode==phecode").reset_index(drop=True)
phecodes_long_mapped["endpoint"] = "phecode_" + phecodes_long_mapped["phecode"]
phecodes_long_mapped

,eid,icd,date,source,phecode,endpoint
0,RID00001,L82,2000-12-15,HES,670,phecode_670
1,RID00001,M47,2001-03-29,HES,715,phecode_715
2,RID00001,M47,2001-03-29,HES,715-1,phecode_715-1
3,RID00001,M47,2001-05-13,HES,715,phecode_715
4,RID00001,M47,2001-05-13,HES,715-1,phecode_715-1
...,...,...,...,...,...,...
236920,RID07411,C18,2018-08-21,ONS,101-4,phecode_101-4
236921,RID07411,C18,2018-08-21,ONS,101-41,phecode_101-41
236922,RID07411,C18,2018-08-21,ONS,130,phecode_130
236923,RID07411,C50,2018-08-21,ONS,105,phecode_105


In [176]:
deaths_long_mapped = deaths[["eid", "date"]].drop_duplicates().assign(source="ONS", icd="", phecode="", endpoint = "OMOP_4306655")[["eid", "icd", "date", "source", "phecode", "endpoint"]]
deaths_long_mapped

,eid,icd,date,source,phecode,endpoint
0,RID02874,,2018-11-11,ONS,,OMOP_4306655
1,RID07411,,2018-08-21,ONS,,OMOP_4306655
2,RID00626,,2018-10-27,ONS,,OMOP_4306655
3,RID07082,,2018-08-12,ONS,,OMOP_4306655
4,RID05386,,2018-11-09,ONS,,OMOP_4306655
...,...,...,...,...,...,...
111,RID06661,,2018-12-20,ONS,,OMOP_4306655
112,RID01844,,2019-02-10,ONS,,OMOP_4306655
113,RID00234,,2018-12-19,ONS,,OMOP_4306655
114,RID02958,,2018-12-21,ONS,,OMOP_4306655


In [177]:
endpoints_long_mapped = pd.concat([phecodes_long_mapped, deaths_long_mapped], axis=0).sort_values(["eid", "date"]).reset_index(drop=True).merge(metadata_md, on="eid", how="left")
endpoints_long_mapped = endpoints_long_mapped[['eid','recruitment_date', "death_date", 'exit_date', 'source', 'date',  'icd', 'phecode', 'endpoint',]]
endpoints_long_mapped

,eid,recruitment_date,death_date,exit_date,source,date,icd,phecode,endpoint
0,RID00001,2007-08-07,NaT,2019-06-17,HES,2000-12-15,L82,670,phecode_670
1,RID00001,2007-08-07,NaT,2019-06-17,HES,2001-03-29,M47,715,phecode_715
2,RID00001,2007-08-07,NaT,2019-06-17,HES,2001-03-29,M47,715-1,phecode_715-1
3,RID00001,2007-08-07,NaT,2019-06-17,HES,2001-05-13,M47,715,phecode_715
4,RID00001,2007-08-07,NaT,2019-06-17,HES,2001-05-13,M47,715-1,phecode_715-1
...,...,...,...,...,...,...,...,...,...
237036,RID07411,2008-06-03,2018-08-21,2018-08-21,ONS,2018-08-21,C18,101-41,phecode_101-41
237037,RID07411,2008-06-03,2018-08-21,2018-08-21,ONS,2018-08-21,C18,130,phecode_130
237038,RID07411,2008-06-03,2018-08-21,2018-08-21,ONS,2018-08-21,C50,105,phecode_105
237039,RID07411,2008-06-03,2018-08-21,2018-08-21,ONS,2018-08-21,C50,130,phecode_130


In [178]:
endpoints_long_mapped.to_feather(f"{base_path}/data/records_long_230320.feather")

In [179]:
records_prev = endpoints_long_mapped.query("date<=recruitment_date")
records_inc = endpoints_long_mapped.query("date>recruitment_date")

In [180]:
data_outcomes_prev = records_prev[["eid", "endpoint"]].drop_duplicates().reset_index(drop=True).assign(prev=1)
data_outcomes_prev

,eid,endpoint,prev
0,RID00001,phecode_670,1
1,RID00001,phecode_715,1
2,RID00001,phecode_715-1,1
3,RID00001,phecode_349,1
4,RID00001,phecode_718,1
...,...,...,...
28542,RID07411,phecode_708-1,1
28543,RID07411,phecode_164,1
28544,RID07411,phecode_105,1
28545,RID07411,phecode_130,1


In [181]:
data_outcomes_event_time = records_inc.sort_values(["eid", "endpoint", "date"]).drop_duplicates(["eid", "endpoint", "date"], keep="first")\
    .assign(event=1, time=lambda x: (x.date - x.recruitment_date).dt.days / 365.25).reset_index(drop=True)[["eid", "endpoint", "event", "time"]]
data_outcomes_event_time

,eid,endpoint,event,time
0,RID00001,phecode_288,1,7.271732
1,RID00001,phecode_322,1,3.482546
2,RID00001,phecode_330,1,3.482546
3,RID00001,phecode_330,1,4.563997
4,RID00001,phecode_330,1,5.900068
...,...,...,...,...
176938,RID07411,phecode_401-1,1,1.952088
176939,RID07411,phecode_520,1,1.952088
176940,RID07411,phecode_520-1,1,1.952088
176941,RID07411,phecode_520-14,1,1.952088


In [182]:
data_outcomes_event = data_outcomes_event_time[["eid", "endpoint", "event"]]
data_outcomes_time = data_outcomes_event_time[["eid", "endpoint", "time"]]

In [183]:
temp_wide = data_outcomes_prev.pivot_table(index='eid', columns='endpoint', values='prev', fill_value=0)
data_outcomes_prev_wide = data_baseline[["eid"]].merge(temp_wide, on="eid", how="left").fillna(0)
data_outcomes_prev_wide.reset_index(drop=True, inplace=True)
data_outcomes_prev_wide = data_outcomes_prev_wide.set_index("eid")
data_outcomes_prev_wide.columns = [f"{c}_prev" for c in data_outcomes_prev_wide.columns]

data_outcomes_prev_wide

,phecode_001_prev,phecode_004_prev,phecode_004-1_prev,phecode_006_prev,phecode_021_prev,phecode_038_prev,phecode_052_prev,phecode_052-1_prev,phecode_052-3_prev,phecode_052-31_prev,...,phecode_840_prev,phecode_840-8_prev,phecode_848_prev,phecode_900_prev,phecode_900-1_prev,phecode_932_prev,phecode_938_prev,phecode_979_prev,phecode_981_prev,phecode_997_prev
eid,,,,,,,,,,,,,,,,,,,,,
RID00001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RID00002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RID00003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RID00004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RID00006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RID07407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RID07408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RID07409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [184]:
temp_wide = data_outcomes_event.pivot_table(index='eid', columns='endpoint', values='event', fill_value=0)
data_outcomes_event_wide = data_baseline[["eid"]].merge(temp_wide, on="eid", how="left").fillna(0)
data_outcomes_event_wide.reset_index(drop=True, inplace=True)
data_outcomes_event_wide = data_outcomes_event_wide.set_index("eid")
data_outcomes_event_wide.columns = [f"{c}_event" for c in data_outcomes_event_wide.columns]

data_outcomes_event_wide

,OMOP_4306655_event,phecode_001_event,phecode_004_event,phecode_004-1_event,phecode_005_event,phecode_005-1_event,phecode_005-2_event,phecode_006_event,phecode_007_event,phecode_007-1_event,...,phecode_840-8_event,phecode_848_event,phecode_891_event,phecode_891-2_event,phecode_893_event,phecode_893-1_event,phecode_979_event,phecode_981_event,phecode_984_event,phecode_997_event
eid,,,,,,,,,,,,,,,,,,,,,
RID00001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RID00002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RID00003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RID00004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RID00006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RID07407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RID07408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RID07409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [185]:
temp_wide = data_outcomes_time.pivot_table(index='eid', columns='endpoint', values='time', fill_value=np.nan)
data_outcomes_time_wide = data_baseline[["eid"]].merge(temp_wide, on="eid", how="left")
data_outcomes_time_wide.reset_index(drop=True, inplace=True)
data_outcomes_time_wide = data_outcomes_time_wide.set_index("eid")

metadata_md = metadata_md.set_index("eid")

# Use the 'apply()' function to replace missing values (NaN) with the corresponding 'exit_time' values
data_outcomes_time_wide = data_outcomes_time_wide.apply(
    lambda row: row.where(
        pd.notnull(row), metadata_md.loc[row.name]["exit_time"]
    ),
    axis=1
)

# Reset the index of 'metadata_md' if needed
metadata_md.reset_index(inplace=True)

data_outcomes_time_wide.columns = [f"{c}_time" for c in data_outcomes_time_wide.columns]

# Check the updated 'data_outcomes_time_wide' DataFrame
data_outcomes_time_wide

,OMOP_4306655_time,phecode_001_time,phecode_004_time,phecode_004-1_time,phecode_005_time,phecode_005-1_time,phecode_005-2_time,phecode_006_time,phecode_007_time,phecode_007-1_time,...,phecode_840-8_time,phecode_848_time,phecode_891_time,phecode_891-2_time,phecode_893_time,phecode_893-1_time,phecode_979_time,phecode_981_time,phecode_984_time,phecode_997_time
eid,,,,,,,,,,,,,,,,,,,,,
RID00001,11.860370,11.860370,11.860370,11.860370,11.860370,11.860370,11.860370,11.860370,11.860370,11.860370,...,11.860370,11.860370,11.860370,11.860370,11.860370,11.860370,11.860370,11.860370,11.860370,11.860370
RID00002,8.941821,8.941821,8.941821,8.941821,8.941821,8.941821,8.941821,8.941821,8.941821,8.941821,...,8.941821,8.941821,8.941821,8.941821,8.941821,8.941821,8.941821,8.941821,8.941821,8.941821
RID00003,12.142368,12.142368,12.142368,12.142368,12.142368,12.142368,12.142368,12.142368,12.142368,12.142368,...,12.142368,12.142368,12.142368,12.142368,12.142368,12.142368,12.142368,12.142368,12.142368,12.142368
RID00004,12.878850,12.878850,12.878850,12.878850,12.878850,12.878850,12.878850,12.878850,12.878850,12.878850,...,12.878850,12.878850,12.878850,12.878850,12.878850,12.878850,12.878850,12.878850,12.878850,12.878850
RID00006,11.937029,11.937029,11.937029,11.937029,11.937029,11.937029,11.937029,11.937029,11.937029,11.937029,...,11.937029,11.937029,11.937029,11.937029,11.937029,11.937029,11.937029,11.937029,11.937029,11.937029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RID07407,12.618754,12.618754,12.618754,12.618754,12.618754,12.618754,12.618754,12.618754,12.618754,12.618754,...,12.618754,12.618754,12.618754,12.618754,12.618754,12.618754,12.618754,12.618754,12.618754,12.618754
RID07408,10.494182,10.494182,10.494182,10.494182,10.494182,10.494182,10.494182,10.494182,10.494182,10.494182,...,10.494182,10.494182,10.494182,10.494182,10.494182,10.494182,10.494182,10.494182,10.494182,10.494182
RID07409,11.720739,11.720739,11.720739,11.720739,11.720739,11.720739,11.720739,11.720739,11.720739,11.720739,...,11.720739,11.720739,11.720739,11.720739,11.720739,11.720739,11.720739,11.720739,11.720739,11.720739


In [186]:
data_outcomes_wide = data_outcomes_prev_wide\
    .merge(data_outcomes_event_wide, how="left", left_index=True, right_index=True)\
    .merge(data_outcomes_time_wide, how="left", left_index=True, right_index=True).sort_index(axis=1)
data_outcomes_wide

,OMOP_4306655_event,OMOP_4306655_time,phecode_001_event,phecode_001_prev,phecode_001_time,phecode_004-1_event,phecode_004-1_prev,phecode_004-1_time,phecode_004_event,phecode_004_prev,...,phecode_979_prev,phecode_979_time,phecode_981_event,phecode_981_prev,phecode_981_time,phecode_984_event,phecode_984_time,phecode_997_event,phecode_997_prev,phecode_997_time
eid,,,,,,,,,,,,,,,,,,,,,
RID00001,0.0,11.860370,0.0,0.0,11.860370,0.0,0.0,11.860370,0.0,0.0,...,0.0,11.860370,0.0,0.0,11.860370,0.0,11.860370,0.0,0.0,11.860370
RID00002,0.0,8.941821,0.0,0.0,8.941821,0.0,0.0,8.941821,0.0,0.0,...,0.0,8.941821,0.0,0.0,8.941821,0.0,8.941821,0.0,0.0,8.941821
RID00003,0.0,12.142368,0.0,0.0,12.142368,0.0,0.0,12.142368,0.0,0.0,...,0.0,12.142368,0.0,0.0,12.142368,0.0,12.142368,0.0,0.0,12.142368
RID00004,0.0,12.878850,0.0,0.0,12.878850,0.0,0.0,12.878850,0.0,0.0,...,0.0,12.878850,0.0,0.0,12.878850,0.0,12.878850,0.0,0.0,12.878850
RID00006,0.0,11.937029,0.0,0.0,11.937029,0.0,0.0,11.937029,0.0,0.0,...,0.0,11.937029,0.0,0.0,11.937029,0.0,11.937029,0.0,0.0,11.937029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RID07407,0.0,12.618754,0.0,0.0,12.618754,0.0,0.0,12.618754,0.0,0.0,...,0.0,12.618754,0.0,0.0,12.618754,0.0,12.618754,0.0,0.0,12.618754
RID07408,0.0,10.494182,0.0,0.0,10.494182,0.0,0.0,10.494182,0.0,0.0,...,0.0,10.494182,0.0,0.0,10.494182,0.0,10.494182,0.0,0.0,10.494182
RID07409,0.0,11.720739,0.0,0.0,11.720739,0.0,0.0,11.720739,0.0,0.0,...,0.0,11.720739,0.0,0.0,11.720739,0.0,11.720739,0.0,0.0,11.720739


In [187]:
data_outcomes_wide.reset_index().to_feather(f"{base_path}/data/data_outcomes_wide_230320.feather")

In [188]:
endpoints = sorted(list(set([c.replace("_prev", "").replace("_event", "").replace("_time", "") for c in data_outcomes_wide.columns])))

In [ ]:
endpoints

In [190]:
dfs_long = []
for e in tqdm(endpoints):
    cols = [c for c in data_outcomes_wide.columns if c in [f"{e}_prev", f"{e}_event", f"{e}_time"]]
    temp = data_outcomes_wide[cols]
    cols_post = temp.columns = [c.replace(f"{e}_", "") for c in temp.columns]
    temp = temp.assign(endpoint = e)[["endpoint"] + cols_post]
    dfs_long.append(temp)

  0%|          | 0/833 [00:00<?, ?it/s]

In [191]:
data_outcomes_long_pre = pd.concat(dfs_long, axis=0).reset_index()
data_outcomes_long_pre["prev"] = data_outcomes_long_pre["prev"].fillna(0.0)

metadata_long = data_outcomes_long_pre.query("event==0")[["eid", "event", "time"]].drop_duplicates().reset_index(drop=True)
merged_df = data_outcomes_long_pre.merge(metadata_long, on="eid", how="left", suffixes=("_outcomes", "_metadata"))
merged_df["event"] = merged_df["event_outcomes"].fillna(merged_df["event_metadata"])
merged_df["time"] = merged_df["time_outcomes"].fillna(merged_df["time_metadata"])
merged_df.drop(["event_metadata", "time_metadata", "event_outcomes", "time_outcomes"], axis=1, inplace=True)
data_outcomes_long = merged_df[["eid", "endpoint", "prev", "event", "time"]]

In [192]:
data_outcomes_long.to_feather(f"{base_path}/data/data_outcomes_long_230320.feather")

In [193]:
data_outcomes_long.sample(10)

,eid,endpoint,prev,event,time
5202785,RID06104,phecode_703-1,0.0,0.0,10.264203
728691,RID03976,phecode_110,0.0,0.0,11.800137
1961939,RID05088,phecode_322,0.0,0.0,10.787132
503797,RID03767,phecode_102-5,0.0,0.0,9.746749
4160196,RID07252,phecode_554,0.0,0.0,11.991786
951981,RID02545,phecode_136,0.0,0.0,13.253936
2891181,RID06620,phecode_410,0.0,0.0,7.608487
4027461,RID04926,phecode_530,0.0,0.0,8.487337
2280034,RID04253,phecode_341,0.0,0.0,12.605065
2267123,RID05875,phecode_339,0.0,0.0,12.093087


In [194]:
len(data_outcomes_long)/7411

814.6787208204021